In [19]:
import ast
import os
import pandas as pd
import platform
import re
import selenium
import time
import unidecode

from time import time, sleep
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from unidecode import unidecode
from webdriver_manager.chrome import ChromeDriverManager

In [20]:
mdf = pd.read_csv('musicas.csv')
mdf.sample(5)

,indice_musica,titulo_musica,nome_artista,artistas_participacao,letra,album,data_lancamento,tags,identificador
87,87,Te amar é massa demais,ANAVITÓRIA,[],"[Letra de ""Te amar é massa demais"" com ANAVITÓ...",COR (2021),"Jan. 1, 2021","Pop,Brasil,Em Português,MPB (Música Popular Br...",Te amar é massa demais ANAVITÓRIA
911,911,Logo Agora,Emílio Santiago,[],"Agora, justamente agora\nAgoraque penso em ir ...",Novo Millennium,1 Viewer,"Pop,Brasil,MPB (Música Popular Brasileira),Em ...",Logo Agora Emílio Santiago
97,97,Tempo de Pipa,Cícero Rosa Lins,[],[Verso 1: Cícero]\nQuando você vem ou não?\nO ...,Canções de Apartamento (2011),"Jun. 26, 2011","Rock,Em Português,MPB (Música Popular Brasilei...",Tempo de Pipa Cícero Rosa Lins
116,116,Absurdo 8,"Rodrigo Campos, Juçara Marçal & Gui Amabis","['Gui\xa0Amabis', 'Juçara\xa0Marçal', 'Rodrigo...","Diz, quem nasce da costela\n\nE boia na banhei...",Sambas do Absurdo (2017),"Feb. 9, 2017","Pop,Em Português,MPB (Música Popular Brasileir...","Absurdo 8 Rodrigo Campos, Juçara Marçal & Gui ..."
669,669,Nu Com a Minha Música,Caetano Veloso,[],[Verso 1]\nPenso em ficar quieto um pouquinho ...,Outras Palavras (1981),1981,"Pop,MPB (Música Popular Brasileira),Brasil,Em ...",Nu Com a Minha Música Caetano Veloso


In [21]:
# Quantidade de artistas distintos no CSV

mdf['nome_artista'].nunique()

245

In [22]:
# Função que recebe um nome e retorna o link do perfil do artista correspondente no Genius 

def normaliza_nome_artista(nome_artista):
    nome_artista = nome_artista.strip().replace(u'\xa0', ' ')
    nome_decodificado = unidecode(nome_artista)
    nome_minusculo = nome_decodificado.lower()
    nome_normalizado = re.sub(r'[^a-zA-Z0-9\s]+', '', nome_minusculo)

    return nome_normalizado

def get_link_musicas_artista(nome_artista):
    nome_para_link = nome_artista.lower().capitalize().replace(' ', '-')
    link_musicas_artista = f'https://genius.com/artists/{nome_para_link}/songs'
    
    return link_musicas_artista

In [23]:
# Criando um DF apenas com o nome dos artistas (principais e participações) existentes

df_artistas = pd.DataFrame(mdf['nome_artista'].unique(), columns = ['nome_artista'])
df_artistas['nome_artista'] = df_artistas['nome_artista'].str.split('&')
df_artistas = df_artistas.explode('nome_artista')
df_artistas['nome_artista'] = df_artistas['nome_artista'].apply(normaliza_nome_artista)
df_artistas.reset_index(drop = True, inplace = True)

# Adicionando ao DF o link do perfil de cada artista no Genius 

df_artistas['link_musicas_artista'] = df_artistas['nome_artista'].apply(get_link_musicas_artista)
df_artistas = df_artistas.drop_duplicates(subset=['nome_artista'])

In [28]:
# Tamanho do DataFrame com nomes e páginas dos artistas

len(df_artistas)

237

In [24]:
# Exibe novo DataFrame dos artistas

df_artistas.head(35)

,nome_artista,link_musicas_artista
0,anavitoria,https://genius.com/artists/Anavitoria/songs
1,milton nascimento,https://genius.com/artists/Milton-nascimento/s...
2,chico buarque,https://genius.com/artists/Chico-buarque/songs
3,silva,https://genius.com/artists/Silva/songs
4,anitta,https://genius.com/artists/Anitta/songs
5,ze ramalho,https://genius.com/artists/Ze-ramalho/songs
6,sergio mendes,https://genius.com/artists/Sergio-mendes/songs
7,vitor kley,https://genius.com/artists/Vitor-kley/songs
8,tim maia,https://genius.com/artists/Tim-maia/songs
9,legiao urbana,https://genius.com/artists/Legiao-urbana/songs


In [25]:
# Para persistir os novos dados e índices de parada

dicionario_musicas = {'indice_artista': [],
                     'indice_musica': [],
                     'titulo_musica': [],
                     'nome_artista': [],
                     'letra': [],
                     'album': [],
                     'data_lancamento': [],
                     'tags': [],
                     'identificador' : []}

musicas_df = pd.DataFrame.from_dict(dicionario_musicas)
nome_arquivo_musicas = 'musicas_artistas.csv'
indice_artista = 0
indice_musica = 0
if os.path.isfile(nome_arquivo_musicas):
    musicas_df = pd.read_csv(nome_arquivo_musicas)
    if len(musicas_df) > 0:
        indice_artista = list(musicas_df['indice_artista'])[-1]
        indice_musica = list(musicas_df['indice_musica'])[-1]
else:
    musicas_df.to_csv(nome_arquivo_musicas, index=False)

print(indice_artista, indice_musica)

237 0


In [26]:
# Exibindo as últimas músicas coletadas

a = pd.read_csv('musicas_artistas.csv')
a.tail(50)

,indice_artista,indice_musica,titulo_musica,nome_artista,letra,album,data_lancamento,tags,identificador
17476,233,138,Lion Man (Instrumental),Criolo,NaN,Nó na Orelha (Instrumental) (2011),"Apr. 25, 2011","Rap,Instrumental,Brasil",Lion Man (Instrumental) Criolo
17477,233,140,Samba Sambei (Instrumental),Criolo,NaN,Nó na Orelha (Instrumental) (2011),"Apr. 25, 2011","Rap,Instrumental,Brasil",Samba Sambei (Instrumental) Criolo
17478,233,141,Sucrilhos (Instrumental),Criolo,NaN,Nó na Orelha (Instrumental) (2011),"Apr. 25, 2011","Rap,Brasil,Instrumental",Sucrilhos (Instrumental) Criolo
17479,233,142,Freguês da Meia Noite (Instrumental),Criolo,NaN,Nó na Orelha (Instrumental) (2011),"Apr. 25, 2011","Rap,Brasil,Instrumental",Freguês da Meia Noite (Instrumental) Criolo
17480,233,144,Dez Anjos,Milton Nascimento & Criolo,[Verso 1: Milton Nascimento]\nTanto barro pra ...,Existe Amor (2020),"May. 8, 2020","Rock,Brasil,MPB (Música Popular Brasileira),Em...",Dez Anjos Milton Nascimento & Criolo
17481,233,147,Coroné Antônio Bento,Criolo,Coroné Antonio Bento\nNo dia do casamento\nDa ...,Viva Tim Maia! (2015),1 Viewer,Pop,Coroné Antônio Bento Criolo
17482,234,0,Primavera (Vai Chuva),Criolo,Quando o inverno chegar\nEu quero estar junto ...,Viva Tim Maia! (2015),1 Viewer,Pop,Primavera (Vai Chuva) Criolo
17483,235,38,De Graça,Marcelo Jeneci,Não venha me dizer\nQue a conta é uma só\nÓÓ Ó...,De Graça (2013),"Oct. 18, 2013","Pop,Em Português",De Graça Marcelo Jeneci
17484,235,39,"Tudo Bem, Tanto Faz",Marcelo Jeneci,Não sai\nPor quê?\nTem tanto e-mail pra respon...,De Graça (2013),"Oct. 18, 2013","Pop,Em Português","Tudo Bem, Tanto Faz Marcelo Jeneci"
17485,235,40,Show de Estrelas,Marcelo Jeneci,"Era uma chuva, era um show de estrelas\nChovia...",Feito Pra Acabar (2010),"Oct. 30, 2010","Pop,Em Português",Show de Estrelas Marcelo Jeneci


In [27]:
# Identificando qual o sistema operacional

sistema_operacional = platform.system()
if sistema_operacional == 'Linux':
    caminho_webdriver = os.path.join('chromedriver_linux64', 'chromedriver')
elif sistema_operacional == 'Windows':
    caminho_webdriver = os.path.join('chromedriver_win64', 'chromedriver.exe')

# Criando o WebDriver
    
servico = Service(executable_path=caminho_webdriver)
opcoes = webdriver.ChromeOptions()
#opcoes.add_argument('--headless')
#opcoes.add_argument('--disable-gpu')
navegador = webdriver.Chrome(service = servico, options = opcoes)
espera_navegador = WebDriverWait(navegador, 30)
identificadores = list(musicas_df['identificador'])

for i in range(indice_artista, len(df_artistas)):
    link_musicas_artista = df_artistas['link_musicas_artista'].iloc[i]
    navegador.get(link_musicas_artista)
    navegador.execute_script("document.body.style.zoom='60%'")

    '''while True:
        script = f"window.scrollTo(0, (document.body.scrollHeight - {900}));"
        navegador.execute_script(script)
        sleep(1)
        if not (len(navegador.find_elements(By.XPATH, "//div[contains(@class, 'PlaceholderSpinner')]"))) > 0:
            break
    
    if len(navegador.find_elements(By.CLASS_NAME, 'render_404-headline')) > 0:
        continue'''
    
    tempo_maximo_espera = 100 
    tempo_inicio = time()
    while True:
        script = f"window.scrollTo(0, (document.body.scrollHeight));"
        navegador.execute_script(script)
        sleep(1)

        elementos_placeholder = navegador.find_elements(By.XPATH, "//div[contains(@class, 'PlaceholderSpinner')]")
        if not elementos_placeholder:
            break
            
        tempo_decorrido = time() - tempo_inicio
        if tempo_decorrido > tempo_maximo_espera:
            print("Tempo de espera excedido. Saindo do loop.")
            break
    
    if len(navegador.find_elements(By.CLASS_NAME, 'render_404-headline')) > 0:
        continue
    
    musicas_elementos = navegador.find_elements(By.XPATH, '//a[contains(@class, "ListItem__Link")]')
    for j in range(indice_musica, len(musicas_elementos)):

        try:
            musica = musicas_elementos[j]
            link_letra = musica.get_attribute("href")
            navegador.execute_script(f'window.open("{link_letra}","_blank");')
            navegador.switch_to.window(navegador.window_handles[1])
        except TimeoutException:
            continue

        # Captura informações importantes da música
        
        titulo_musica = ''
        titulo_musica_elemento = navegador.find_element(By.XPATH, '//span[contains(@class, "SongHeaderdesktop__HiddenMask")]')
        if titulo_musica_elemento:
            titulo_musica = titulo_musica_elemento.get_attribute("innerText")
        
        nome_artista = ''
        nome_artista_elemento = navegador.find_element(By.XPATH, '//a[contains(@class, "HeaderArtistAndTracklistdesktop__Artist")]')
        if nome_artista_elemento:
            nome_artista = nome_artista_elemento.get_attribute("innerText")
        
        identificador = f'{titulo_musica} {nome_artista}'
        
        if identificador not in identificadores:
            
            letra = ''
            try:
                identificador_containers_letra_id = '//div[@data-lyrics-container="true"]'
                containers_letra = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, identificador_containers_letra_id)))
                for container in containers_letra:
                    letra_atual = container.get_attribute("innerText")
                    letra += letra_atual
            except Exception as e:
                print(f'Erro na letra: {str(e)}')

            album = ''
            try:
                album = navegador.find_element(By.XPATH, "//a[contains(@class, 'PrimaryAlbum')]").get_attribute("innerText")
            except Exception as e:
                print(f'Erro no nome do álbum: {str(e)}')

            data_lancamento = ''
            try:
                identificador_data_lancamento = "//span[contains(@class, 'MetadataStats')]"
                elementos_data_lancamento = navegador.find_elements(By.XPATH, identificador_data_lancamento)
                data_lancamento = elementos_data_lancamento[0].get_attribute("innerText")
            except Exception as e:
                print(f'Erro na data de lançamento: {str(e)}')

            tags = ''
            try:
                identificador_tags = "//a[contains(@class, 'SongTags__Tag')]"
                elementos_tags = navegador.find_elements(By.XPATH, identificador_tags)
                tags = ','.join([tag.get_attribute("innerText") for tag in elementos_tags])
            except Exception as e:
                print(f'Erro nas tags: {str(e)}')
            
            i_copia = i
            j_copia = j + 1
            if j ==  len(musicas_elementos) - 1:
                i_copia += 1
                j_copia = 0
                
            nova_linha = [i_copia, j_copia, titulo_musica, nome_artista, letra, album, data_lancamento, tags, identificador]
            df_nova_linha = pd.DataFrame([nova_linha])
            df_nova_linha.to_csv(nome_arquivo_musicas, mode='a', header=False, index=False)
            identificadores.append(identificador)

        janelas = navegador.window_handles
        if len(janelas) > 0:
            navegador.close()
            navegador.switch_to.window(janelas[0])